In [1]:
import pandas as pd 
import numpy as np
import folium
import time

from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# 웹드라이버 설정
chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument("headless")

driver = webdriver.Chrome("../driver/chromedriver.exe", options=chrome_options)
driver.implicitly_wait(10)

# 스타벅스 지도 페이지 오픈
starbuck_map_url = "https://www.starbucks.co.kr/store/store_map.do?disp=locale"
driver.get(starbuck_map_url)

C:\Users\jukim\AppData\Local\Temp\ipykernel_1992\977915328.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("../driver/chromedriver.exe", options=chrome_options)


In [3]:
# 윈도우 화면 최대화
driver.maximize_window()

In [4]:
#스타벅스 서울 클릭
starbucks_seoul_css = "#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_css))).click()


In [5]:
# 스타벅스 서울 전체 클릭
starbucks_seoul_all_css = "#mCSB_2_container > ul > li:nth-child(1) > a"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, starbucks_seoul_all_css))).click()


In [6]:
# 스타벅스 서울 전체 HTML 파싱
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

In [7]:
container = soup.find("div", id = "mCSB_3_container")
li_all = container.find_all("li")

In [8]:
starbucks_data = []

for li in li_all:
    name = li.find("strong").text.strip()
    address = li.find("p").text.strip().replace("1522-3232", "")
    gu = address.split(" ")[1]
    each = {
        "매장이름":name, "주소":address, "구":gu, "브랜드":"스타벅스"
    }
    starbucks_data.append(each)
    

In [9]:
len(starbucks_data)

599

In [10]:
df_starbucks = pd.DataFrame(starbucks_data)
df_starbucks.tail()

,매장이름,주소,구,브랜드
594,사가정역,서울특별시 중랑구 면목로 310,중랑구,스타벅스
595,상봉역,서울특별시 중랑구 망우로 307 (상봉동),중랑구,스타벅스
596,묵동,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",중랑구,스타벅스
597,양원역,서울특별시 중랑구 양원역로10길 3 (망우동),중랑구,스타벅스
598,중화역,서울특별시 중랑구 봉화산로 35,중랑구,스타벅스


In [11]:
gu_list = list(df_starbucks["구"].unique())
len(gu_list)

25

In [15]:
# 이디야 지도 페이지 오픈
eidya_map_url = "https://ediya.com/contents/find_store.html"
driver.get(eidya_map_url)

In [16]:
# 이디야 페이지 화면 최대화

driver.maximize_window()

WebDriverException: Message: unknown error: failed to change window state to 'normal', current state is 'maximized'
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x01196643]
	(No symbol) [0x0112BE21]
	(No symbol) [0x0102DA9D]
	(No symbol) [0x01013995]
	(No symbol) [0x010123D3]
	(No symbol) [0x01010C35]
	(No symbol) [0x010A8CBF]
	(No symbol) [0x0107FD7C]
	(No symbol) [0x01096B09]
	(No symbol) [0x0107FB76]
	(No symbol) [0x010549C1]
	(No symbol) [0x01055E5D]
	GetHandleVerifier [0x0140A142+2497106]
	GetHandleVerifier [0x014385D3+2686691]
	GetHandleVerifier [0x0143BB9C+2700460]
	GetHandleVerifier [0x01243B10+635936]
	(No symbol) [0x01134A1F]
	(No symbol) [0x0113A418]
	(No symbol) [0x0113A505]
	(No symbol) [0x0114508B]
	BaseThreadInitThunk [0x767B7D69+25]
	RtlInitializeExceptionChain [0x7784BB9B+107]
	RtlClearBits [0x7784BB1F+191]


In [ ]:
 # 이디야 주소 검색 클릭
eidya_address_css = "#contentWrap > div.contents > div > div.store_search_pop > ul > li:nth-child(2) > a"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, eidya_address_css))).click()

In [25]:
eidya_data = []

# 이디야 지도 서울 > 지역별 데이터 검색
for gu in tqdm_notebook(gu_list):
    
    eidya_search_keyword_css = "#keyword"
    
    #이디야 주소 검색어 초기화
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, eidya_search_keyword_css ))).clear()

    # 이디야 주소 검색어 입력
   
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, eidya_search_keyword_css ))).send_keys(f"서울 {gu}")

    # 이디야 주소 검색 버튼 클릭
    eidya_search_button_css = "#keyword_div > form > button"
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, eidya_search_button_css ))).click()
    
    #이디야 구장 매장 정보 수집
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    ul_tag = soup.find("ul", id="placesList")
    dl_all = ul_tag.find_all("dl")
    
    for dl in dl_all:
        name = dl.find("dt").text.strip()
        address = dl.find("dd").text.strip()
        gu = address.split(" ")[1]
        each = {
            "매장이름":name, "주소":address, "구":gu, "브랜드":"이디야"
        }
        eidya_data.append(each)

C:\Users\jukim\AppData\Local\Temp\ipykernel_1992\4044251298.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for gu in tqdm_notebook(gu_list):


  0%|          | 0/25 [00:00<?, ?it/s]

In [26]:
len(eidya_data)

724

In [27]:
df_ediya = pd.DataFrame(eidya_data)
df_ediya.tail()

,매장이름,주소,구,브랜드
719,용산청파점,"서울 용산구 청파로71길 10 (청파동1가, 새마을금고)",용산구,이디야
720,용산효창점,서울 용산구 효창원로66길 2 (효창동),용산구,이디야
721,용산후암점,"서울 용산구 후암로 40-2 (후암동, 성원빌딩)",용산구,이디야
722,원효로점,서울 용산구 원효로 257 (원효로1가),용산구,이디야
723,이태원역점,서울 용산구 이태원로 176 (이태원동),용산구,이디야


In [30]:
df_coffee = pd.concat([df_starbucks, df_ediya])
df_coffee.reset_index(drop=True, inplace=True)

In [31]:
df_coffee.tail()

,매장이름,주소,구,브랜드
1318,용산청파점,"서울 용산구 청파로71길 10 (청파동1가, 새마을금고)",용산구,이디야
1319,용산효창점,서울 용산구 효창원로66길 2 (효창동),용산구,이디야
1320,용산후암점,"서울 용산구 후암로 40-2 (후암동, 성원빌딩)",용산구,이디야
1321,원효로점,서울 용산구 원효로 257 (원효로1가),용산구,이디야
1322,이태원역점,서울 용산구 이태원로 176 (이태원동),용산구,이디야


In [33]:
!pip install googlemaps

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40717 sha256=8ed74e32fc2b00cff4014b2911ef81b2d37a67d9f7c17ee34c5f18900b0f3fc9
  Stored in directory: c:\users\jukim\appdata\local\pip\cache\wheels\d9\5f\46\54a2bdb4bcb07d3faba4463d2884865705914cc72a7b8bb5f0
Successfully built googlemaps


### 구글 지도를 이용한 매장의 위치 자표 반환

In [34]:
import googlemaps

In [ ]:
google_maps_key = "AIl"

In [17]:
# 이디야 주소 검색 클릭
eidya_address_css = "#contentWrap > div.contents > div > div.store_search_pop > ul > li:nth-child(2) > a"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, eidya_address_css))).click()

In [18]:
# 이디야 주소 검색어 입력
eidya_search_keyword_css = "#keyword"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, eidya_search_keyword_css ))).send_keys("서울 강남구")

In [19]:
# 이디야 주소 검색 버튼 클릭
eidya_search_button_css = "#keyword_div > form > button"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, eidya_search_button_css ))).click()